In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import re
from io import StringIO
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
data = """,Date,Venue
0,10/09/2023,ST
1,13/09/2023,HV
2,17/09/2023,ST
3,20/09/2023,HV
4,24/09/2023,ST
5,27/09/2023,HV
6,01/10/2023,ST
7,04/10/2023,HV
8,08/10/2023,ST
9,11/10/2023,HV
10,15/10/2023,ST
11,18/10/2023,HV
12,22/10/2023,ST
13,25/10/2023,ST
"""

# Read into DataFrame
df = pd.read_csv(StringIO(data))
print(df)

    Unnamed: 0        Date Venue
0            0  10/09/2023    ST
1            1  13/09/2023    HV
2            2  17/09/2023    ST
3            3  20/09/2023    HV
4            4  24/09/2023    ST
5            5  27/09/2023    HV
6            6  01/10/2023    ST
7            7  04/10/2023    HV
8            8  08/10/2023    ST
9            9  11/10/2023    HV
10          10  15/10/2023    ST
11          11  18/10/2023    HV
12          12  22/10/2023    ST
13          13  25/10/2023    ST


In [26]:
params = {
        "RaceDate": date,
        "Racecourse": venue,
        "RaceNo": race_no
    }
print(params)

{'RaceDate': '25/10/2023', 'Racecourse': 'ST', 'RaceNo': 1}


In [27]:
response = requests.get(base_url, params=params, headers=headers, verify=False)

In [28]:
soup = BeautifulSoup(response.text, 'html.parser')

In [29]:
table = soup.find("table", class_ = "f_tac table_bd draggable")

In [35]:
headers = [th.get_text(strip = True) for th in table.find('tr').find_all('td')]

In [36]:
headers

['Pla.',
 'Horse No.',
 'Horse',
 'Jockey',
 'Trainer',
 'Act. Wt.',
 'Declar. Horse Wt.',
 'Dr.',
 'LBW',
 'RunningPosition',
 'Finish Time',
 'Win Odds']

In [30]:
table

<table class="f_tac table_bd draggable" style="width: 760px;">
<thead>
<tr class="bg_blue color_w">
<td style="width: 50px;" title="Placing">Pla.</td>
<td style="width: 60px;">Horse No.</td>
<td style="width: 245px;">Horse</td>
<td style="width: 125px;">Jockey</td>
<td style="width: 125px;">Trainer</td>
<td style="width: 55px;" title="Actual Weight">Act. Wt.</td>
<td style="width: 55px;" title="On Date Horse Weight">Declar. Horse Wt.</td>
<td style="width: 55px;" title="Draw">Dr.</td>
<td style="width: 55px;" title="Length Behind Winner">LBW</td>
<td style="width: 70px;">Running<br/>Position</td>
<td style="width: 65px;">Finish Time</td>
<td style="width: 65px;">Win Odds</td>
</tr>
</thead>
<tbody class="f_fs12">
<tr>
<td style="white-space: nowrap;"><a href="//racing.hkjc.com/racing/finishlinephoto/english/finishphoto.asp?path=/racing/common/images/finishphoto/23_24/20231025/20231025R01_P1.jpg" onclick="WATracker.trackClickEvent('btn_RacingLocalResult_ResultPhoto_20231025R01_P1.jpg_En

In [4]:
for idx, row in df.iterrows():
    date = row['Date']
    venue = row['Venue']

In [5]:
date

'25/10/2023'

In [6]:
venue

'ST'

In [7]:
max_race_no = 12
race_no = range(1, max_race_no + 1)

In [11]:
base_url = "https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx"
http_headers = {"User-Agent": "Mozilla/5.0 (compatible; HKJCScraper/1.0)"}
all_links_set = set()  # use set to avoid duplicates

In [12]:
for race_no in range(1, max_race_no + 1):
    params = {
        "RaceDate": date,
        "Racecourse": venue,
        "RaceNo": race_no
    }

    print(params)

    response = requests.get(base_url, params=params, headers=http_headers, verify=False)

    if response.status_code != 200:
        print(f"  Race {race_no}: Failed to fetch (status {response.status_code}). Stopping.")
        break # breaking the loop of race no

    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.find("table", class_ = "f_tac table_bd draggable")
    if not table:
        print(f'table not found')
        break

    header_row = table.find('tr')
    headers = [td.get_text(strip = True) for td in header_row.find_all('td')]
    if 'Horse' not in headers:
        print("no 'horse' column found")
        break

    # get position of the horse column
    horse_col_index = headers.index('Horse')

    race_links = []

    for row in table.find_all('tr')[1:]:  # skip header row
        cells = row.find_all('td') # get all cells in the row
        if len(cells) > horse_col_index:
            td = cells[horse_col_index]
            a_tag = td.find('a', href = True)
            if a_tag:
                href = a_tag['href']
                if href.startswith('/'):
                    href = "https://racing.hkjc.com" + href
                race_links.append(href) 

    print(race_links)


{'RaceDate': '25/10/2023', 'Racecourse': 'ST', 'RaceNo': 1}
['https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2021_G029', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2021_G279', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H022', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2020_E409', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2019_D157', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2021_G301', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2020_E151', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2019_D070', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2020_E316', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2021_G470', 'https://racing.hkj

In [22]:
race_links

['https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H246',
 'https://racing.hkjc.com/racing/information/English/Jockey/JockeyProfile.aspx?JockeyId=AA&Season=Current',
 'https://racing.hkjc.com/racing/information/English/Trainers/TrainerProfile.aspx?TrainerId=SCS&Season=Current',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H141',
 'https://racing.hkjc.com/racing/information/English/Jockey/JockeyProfile.aspx?JockeyId=AVB&Season=Current',
 'https://racing.hkjc.com/racing/information/English/Trainers/TrainerProfile.aspx?TrainerId=RW&Season=Current',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H337',
 'https://racing.hkjc.com/racing/information/English/Jockey/JockeyProfile.aspx?JockeyId=PZ&Season=Current',
 'https://racing.hkjc.com/racing/information/English/Trainers/TrainerProfile.aspx?TrainerId=MKL&Season=Current',
 'https://racing.hkjc.com/racing/information/English/Horse

In [ ]:
def scrape_all_horse_links(fixtures_df, max_race_no=12):
    
    base_url = "https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx"
    headers = {"User-Agent": "Mozilla/5.0 (compatible; HKJCScraper/1.0)"}
    all_links_set = set()  # use set to avoid duplicates

    # loop for date and venue
    for idx, row in fixtures_df.iterrows():
        date_str = row['Date']
        venue = row['Venue']

        print(f"Processing meeting: {date_str} @ {venue}")

        # loop for race numbers
        for race_no in range(1, max_race_no + 1):
            params = {
                "RaceDate": date_str,
                "Racecourse": venue,
                "RaceNo": race_no
            }
            response = requests.get(base_url, params=params, headers=headers, verify=False)
            if response.status_code != 200:
                print(f"  Race {race_no}: Failed to fetch (status {response.status_code}). Stopping.")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            table = soup.find('table', class_='table_bd')

            if not table:
                print(f"  Race {race_no}: No result table. Stopping.")
                break  # Stop checking further races for this meeting

            # Extract horse links from 'horses' column
            race_links = []
            for td in table.find_all('td', class_='horses'):
                a_tag = td.find('a', href=True)
                if a_tag:
                    href = a_tag['href']
                    if href.startswith('/'):
                        href = "https://racing.hkjc.com" + href
                    race_links.append(href)

            # Add to the set to keep unique links
            all_links_set.update(race_links)
            print(f"  Race {race_no}: Found {len(race_links)} horse links (Total so far: {len(all_links_set)})")

    return list(all_links_set)


In [ ]:
links = scrape_all_horse_links(df)

Processing meeting: 10/09/2023 @ ST


c:\Users\TCL0011\AppData\Local\anaconda3\envs\ml1\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'racing.hkjc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\TCL0011\AppData\Local\anaconda3\envs\ml1\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'racing.hkjc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
